In [1]:
# ==============================================================================
# @title NOTEBOOK 1: THE TRAINING GROUND
# "Graduating the AIT Physicist"
# ==============================================================================
#
# MISSION:
# To create a "Universal Diagnostic Agent" (The AIT Physicist) capable of
# recognizing the underlying generative grammar of dynamic systems.
#
# METHODOLOGY:
# 1. The Generator: Simulate the "Wolfram Computational Universe" (ECA rules).
# 2. The Architecture: Construct a Tiny Recursive Model (TRM) optimized for
#    causal structure.
# 3. The Calibration: Train on the "Prime 9" rules derived from Riedel & Zenil.
# 4. The Verification: Test on the "Riedel Composition" to prove emergence detection.
#
# OUTPUT:
# - 'trm_expert.pth': The frozen weights of our scientific instrument.
# ==============================================================================

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive

# --- CONFIGURATION ---
CONFIG = {
    'WIDTH': 4,             # The "Lattice Size" (Matches our 4-bin market encoder)
    'WINDOW': 30,           # The "Time Horizon" (How far back we look)
    'HIDDEN_DIM': 64,       # Brain size (Occam's Razor)
    'EPOCHS': 100,          # Training duration
    'BATCH_SIZE': 128,      # Learning throughput
    'LR': 0.001             # Learning rate
}

# The "Prime 9" - Scientifically grounded in Riedel & Zenil (2018)
PRIME_RULES = [0, 15, 30, 54, 60, 90, 110, 170, 254]
RULE_MAP = {r: i for i, r in enumerate(PRIME_RULES)}

# ==============================================================================
# 1. THE UNIVERSE GENERATOR (Physics Engine)
# ==============================================================================
def generate_eca_trajectory(rule_number, steps, width):
    """Simulates an Elementary Cellular Automaton."""
    rule_bin = np.array([int(x) for x in np.binary_repr(rule_number, width=8)], dtype=np.uint8)
    current_state = np.random.randint(2, size=width)
    trajectory = [current_state]

    for _ in range(steps - 1):
        left = np.roll(current_state, 1)
        right = np.roll(current_state, -1)
        neighborhood = 4 * left + 2 * current_state + 1 * right
        next_state = rule_bin[7 - neighborhood]
        trajectory.append(next_state)
        current_state = next_state
    return np.array(trajectory)

def encode_trajectory_to_market_format(traj):
    """
    Maps binary ECA (0/1) to the 4-bin One-Hot format used by the Market Encoder.
    0 -> Bin 0 [1,0,0,0] (Low Energy)
    1 -> Bin 3 [0,0,0,1] (High Energy)
    This aligns "Active Cells" with "High Acceleration".
    """
    # Create (Time, Width, 4) array
    # But wait, our TRM expects (Time, 4).
    # Our market encoder outputs a 4-wide vector per time step.
    # So we map the scalar ECA state to a vector.
    # Actually, the TRM takes (Batch, Time, Input_Width).
    # Market Input Width = 4 (One-hot bins).
    # So we need to map ECA state (scalar 0/1) to a 4-dim vector?
    # NO. The market encoder bins *acceleration* into 4 bins.
    # A single time step in market data is a 4-dim one-hot vector.
    # To match this, we treat the ECA width as the "Batch" or just train on 1D streams?

    # CORRECTION: The TRM is applied to a window of *one asset*.
    # Market Data: (Time, 4_bins).
    # ECA Data: The generator makes a grid of (Time, Space).
    # We should treat *each column* of the ECA grid as a separate sample time series.
    # And we map the 0/1 state of that cell to a 4-bin vector.

    T, W = traj.shape
    # Reshape to (Time * Space) samples of length 1? No.
    # We want (Samples, Window, 4).

    # Let's generate single-cell time series.
    # State 0 -> [1, 0, 0, 0]
    # State 1 -> [0, 0, 0, 1]

    one_hot = np.zeros((T, W, 4))
    one_hot[traj == 0, 0] = 1
    one_hot[traj == 1, 3] = 1

    # We transpose to get (W, T, 4) -> List of time series
    return one_hot.transpose(1, 0, 2)

# ==============================================================================
# 2. THE ARCHITECTURE (Tiny Recursive Model)
# ==============================================================================
class TinyRecursiveModel(nn.Module):
    def __init__(self, input_width, hidden_dim, num_classes):
        super(TinyRecursiveModel, self).__init__()
        self.encoder = nn.Linear(input_width, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, num_classes)
        )

    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        _, h_n = self.rnn(encoded)
        final_state = h_n.squeeze(0)
        logits = self.head(final_state)
        return logits

# ==============================================================================
# 3. THE CURRICULUM (Data Preparation)
# ==============================================================================
def create_curriculum(num_samples_per_rule=500):
    X_data = []
    y_data = []

    print(f"Generating {num_samples_per_rule} samples per Prime Rule...")
    for rule in tqdm(PRIME_RULES):
        label = RULE_MAP[rule]
        # We generate a wide lattice, then split it into individual time series
        # This ensures we get diverse initial conditions
        traj = generate_eca_trajectory(rule, CONFIG['WINDOW'], num_samples_per_rule)

        # Convert to (Samples, Time, 4)
        series_list = encode_trajectory_to_market_format(traj)

        for s in series_list:
            X_data.append(s)
            y_data.append(label)

    return TensorDataset(
        torch.FloatTensor(np.array(X_data)),
        torch.LongTensor(np.array(y_data))
    )

# ==============================================================================
# 4. EXECUTION (Training)
# ==============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"--- Initializing AIT Physicist on {device} ---")

# 1. Create Data
train_data = create_curriculum(num_samples_per_rule=200)
train_loader = DataLoader(train_data, batch_size=CONFIG['BATCH_SIZE'], shuffle=True)

# 2. Initialize Model
physicist = TinyRecursiveModel(CONFIG['WIDTH'], CONFIG['HIDDEN_DIM'], len(PRIME_RULES)).to(device)
optimizer = optim.Adam(physicist.parameters(), lr=CONFIG['LR'])
criterion = nn.CrossEntropyLoss()

# 3. Train
print("--- Beginning Training Loop ---")
for epoch in tqdm(range(CONFIG['EPOCHS'])):
    physicist.train()
    epoch_loss = 0
    for X_b, y_b in train_loader:
        X_b, y_b = X_b.to(device), y_b.to(device)
        optimizer.zero_grad()
        logits = physicist(X_b)
        loss = criterion(logits, y_b)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}: Loss = {epoch_loss / len(train_loader):.4f}")

print("✅ Training Complete.")

# ==============================================================================
# 5. VERIFICATION (Composition Test)
# ==============================================================================
# Testing Riedel's Theorem: [170, 15, 118] -> Rule 110
print("\n--- Running The Composition Stress Test ---")

def apply_rule(state, rule):
    width = len(state)
    rule_bin = np.array([int(x) for x in np.binary_repr(rule, width=8)], dtype=np.uint8)
    left = np.roll(state, 1); right = np.roll(state, -1)
    neighborhood = 4 * left + 2 * state + 1 * right
    return rule_bin[7 - neighborhood]

def generate_composite_trajectory(rules_chain, steps, width):
    current_state = np.random.randint(2, size=width)
    trajectory = [current_state]
    for _ in range(steps - 1):
        temp_state = current_state.copy()
        for r in rules_chain:
            temp_state = apply_rule(temp_state, r)
        trajectory.append(temp_state)
        current_state = temp_state
    return np.array(trajectory)

# Generate Data
comp_chain = [170, 15, 118] # Corrected Order
raw_traj = generate_composite_trajectory(comp_chain, CONFIG['WINDOW'], 200)
X_comp = encode_trajectory_to_market_format(raw_traj)
X_comp_tensor = torch.FloatTensor(X_comp).to(device)

# Diagnose
physicist.eval()
with torch.no_grad():
    logits = physicist(X_comp_tensor)
    preds = torch.argmax(logits, dim=1).cpu().numpy()

# Count
counts = np.bincount(preds, minlength=len(PRIME_RULES))
idx_110 = RULE_MAP[110]
idx_54 = RULE_MAP[54]
class_4_hits = counts[idx_110] + counts[idx_54]

print(f"Composition {comp_chain} (Should be Rule 110)")
print(f"Class 4 Detection Rate: {class_4_hits / 200:.1%}")

if class_4_hits > 100:
    print("✅ SUCCESS: Emergent Complexity Detected.")
else:
    print("❌ FAILURE: Model failed composition test.")

# ==============================================================================
# 6. ARCHIVING (Save to Drive)
# ==============================================================================
print("\n--- Archiving Protocol ---")
drive.mount('/content/drive', force_remount=True)

ROOT_DIR = '/content/drive/MyDrive/Algoplexity_Research'
PROJECT_DIR = os.path.join(ROOT_DIR, 'Horizon_1_Computational_Phase_Transition')
MODELS_DIR = os.path.join(PROJECT_DIR, 'models')

if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)
    print(f"🆕 Created: {MODELS_DIR}")

save_path = os.path.join(MODELS_DIR, 'trm_expert.pth')
torch.save(physicist.state_dict(), save_path)
print(f"✅ Model saved to: {save_path}")


--- Initializing AIT Physicist on cuda ---
Generating 200 samples per Prime Rule...


  0%|          | 0/9 [00:00<?, ?it/s]

--- Beginning Training Loop ---


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0: Loss = 2.1846
Epoch 20: Loss = 1.4252
Epoch 40: Loss = 1.1688
Epoch 60: Loss = 1.1015
Epoch 80: Loss = 1.0501
✅ Training Complete.

--- Running The Composition Stress Test ---
Composition [170, 15, 118] (Should be Rule 110)
Class 4 Detection Rate: 79.0%
✅ SUCCESS: Emergent Complexity Detected.

--- Archiving Protocol ---
Mounted at /content/drive
✅ Model saved to: /content/drive/MyDrive/Algoplexity_Research/Horizon_1_Computational_Phase_Transition/models/trm_expert.pth
